<a href="https://colab.research.google.com/github/DerWeiseTeufel/Tinkoff_Risk_Analytics_2024/blob/main/HW_4_Khamazaev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Домашнее задание №4. Работа с NPV моделью.

*Дата выдачи: 17.04.2024*

*Дедлайн: 08.05.2024 23:59*

### Задание №1. Построить распределение клиентов по кривым.
- Использовать DEF, CLO, DLQ кривые из прошлой домашки
- Интерполировать и экстраполировать кривые для каждого значения PA/PD
- PA – x_score из вашей домашки


### Задание №2. Добавить страховку в NPV модель.
- Страховка стоит 0.5% от лимита
- Подключена у 40% клиентов в статусе CUR и у 60% в статусе DLQ
- Возможно наступление страхового случая с вероятностью 0.1%, при подключенной страховке мы должны возместить всю сумму текущей задолженности


Вам нужно внести изменения в код ниже. Где – подсказывать не будем, это часть задания. Добавьте комментарий с указанием места, котороые вы поменяли.

### Класс модели

In [2]:
def interpol(bucket_, delta_def_ratio, delta_pd, avg_pd, pd):
	return bucket_ + (delta_def_ratio/delta_pd) * (pd - avg_pd) # формула интерполяции

In [3]:
import numpy as np
def calc_interpolation(bucket_1, bucket_2, bucket_3,avg_pd_1,avg_pd_2,avg_pd_3, pd):
  #avg_pd_1 > avg_pd_2> avg_pd_3 должно быть для того чтобы работало


  buckets = sorted([(avg_pd_1, bucket_1), (avg_pd_2, bucket_2), (avg_pd_3, bucket_3)], reverse=True) # это чтобы было справедливо avg_pd_1 > avg_pd_2> avg_pd_3
  avg_pd_1 = buckets[0][0]
  bucket_1 = buckets[0][1]
  avg_pd_2 = buckets[1][0]
  bucket_2 = buckets[1][1]
  avg_pd_3 = buckets[2][0]
  bucket_3 = buckets[2][1]

  if pd > avg_pd_1:
    delta_def_ratio = bucket_1 - bucket_2
    delta_pd = avg_pd_1 - avg_pd_2
    avg_pd = avg_pd_1
    bucket = bucket_1

  if avg_pd_1>=pd>avg_pd_2:
    delta_def_ratio = bucket_1 - bucket_2
    delta_pd = avg_pd_1 - avg_pd_2
    avg_pd = avg_pd_2
    bucket = bucket_2


  if avg_pd_2>=pd>avg_pd_3:
    delta_def_ratio = bucket_2 - bucket_3
    delta_pd = avg_pd_2 - avg_pd_3
    avg_pd = avg_pd_3
    bucket = bucket_3

  if avg_pd_3>=pd:
    delta_def_ratio = bucket_2 - bucket_3
    delta_pd = avg_pd_2 - avg_pd_3
    avg_pd = avg_pd_3
    bucket = bucket_3

  def_ratio = np.zeros(101)

  for t in range(1,25):
    def_ratio[t] = interpol(bucket[t-1], delta_def_ratio[t-1], delta_pd, avg_pd, pd)

  return def_ratio

In [7]:
def calc_dist(DefRate, CloRate):
  ACT_prev = 1 #доля всех кредиторов на 0 месяц
  DEF_prev = 0
  CLO_prev = 0
  DEF_dist = np.zeros(101)
  CLO_dist = np.zeros(101)
  for i in range(1, 25):
    DEF_i = DefRate[i]*ACT_prev + DEF_prev
    CLO_i = CloRate[i]*ACT_prev + CLO_prev
    ACT_i = 1 - DEF_i - CLO_i

    DEF_dist[i] = DEF_i
    CLO_dist[i] = CLO_i
    DEF_prev = DEF_i
    CLO_prev = CLO_i
    ACT_prev = ACT_i
  return DEF_dist, CLO_dist


In [1]:
import numpy as np
import pandas as pd

class NPVModel:

    avg_missed_payments = 1.5
    recovery = 0.50
    dlq_penalty_amount = 500
    oper_costs = 100
    collection_costs = 600
    tax_rate = 0.20
    discounting_rate = 0.30
    eq_req = 0.125
    cost_of_funds = 0.16
    dlq_ratio = "not assigned yet" # на всякий чтобы ошибку вывело, правда ариф операции и со str работают )
    acquisition_cost = 1000
    insurance_rate = 0.5/100


    def __init__(self):
        pass

    def model_balance_calculations(self, amount, rate, term):
        '''
        Расчет модельных баланса, выплаченных процентов, регулярного платежа
        :param amount: Сумма кредита
        :param rate: Ставка
        :param term: Срок
        :return:
        balance : np.array(101) : остаток тела долга по кредиту на каждый месяц
        interest : np.array(101) : выплата по процентам каждый месяц
        regular_payment : float : размер регулярного платежа
        '''

        regular_payment = round(amount * (rate / 12) *\
                                (1 + (rate / 12)) / (1 - ((1 + rate / 12) ** (-term))))
        # График баланса и процентов
        balance = np.zeros(101)
        interest = np.zeros(101)
        balance[0] = amount

        for i in range(1, term + 1):
            int_payment = balance[i - 1] * rate / 12
            debt_payment = regular_payment - int_payment # насколько уменьшается тело долга
            balance[i] = max(0, round(balance[i - 1] - debt_payment))
            interest[i] = int_payment
        return balance, interest, regular_payment

    def distribution_calc(self, PD, pa, term, path_def_rate,  path_clo_rate, path_dlq_ratio):
        '''
        Расчет распределения по статусам
        :param pd: вероятность выпасть к дефолт к 12-му месяцу
        :param pa: вероятность полного досрочного погашения к 6-му месяцу
        :param term: срок кредита
        :return:
        act_dist : np.array(101) : доля клиентов в статусе ACT на каждый месяц
        dlq_dist : np.array(101) : доля клиентов в статусе CUR на каждый месяц
        cur_dist : np.array(101) : доля клиентов в статусе DLQ на каждый месяц
        def_dist : np.array(101) : доля клиентов в статусе DEF на каждый месяц
        '''

        #подгрузка df
        def_rate = pd.read_csv(path_def_rate)
        clo_rate = pd.read_csv(path_clo_rate)
        dlqratio = pd.read_csv(path_dlq_ratio)
        avg_pd_1 = 0.011040# средний pd по  pd_bucket = 0
        avg_pd_2 = 0.032685 # средний pd по  pd_bucket = 1
        avg_pd_3 = 0.068209# средний pd по pd_bucket = 2
        avg_pa_1 = 0.133270# средний pd по  x_bucket = 0
        avg_pa_2 = 0.220838# средний pd по  x_bucket = 1
        avg_pa_3 = 0.379452# средний pd по  x_bucket = 2
        def_rate_inter = calc_interpolation(def_rate["0"], def_rate["1"], def_rate["2"],avg_pd_1,avg_pd_2,avg_pd_3, PD)# интерполяция для PD
        clo_rate_inter = calc_interpolation(clo_rate["0"], clo_rate["1"], clo_rate["2"],avg_pa_1,avg_pa_2,avg_pa_3, pa) # интерполяция для PA
        def_dist, clo_dist = calc_dist(def_rate_inter, clo_rate_inter) # подсчет распределений
        clo_dist[24:] = clo_dist[24] # после 24 платежа доля clo const
        #def_dist[25:] = 1 - clo_dist[25:] #
        def_dist[24:] = def_dist[24] # после 24 платежа доля def - const
        def_dist[term+1:] = 1 - clo_dist[term] # после окончания срока все кто не в clo , те в def
        clo_dist[term+1:] = clo_dist[term]


        act_dist = 1 - clo_dist - def_dist

        self.dlq_ratio = calc_interpolation(dlqratio["0"], dlqratio["1"],dlqratio["2"],avg_pd_1,avg_pd_2,avg_pd_3, PD) # интерполяция dlq_ratio по PD
        dlq_dist = act_dist*self.dlq_ratio
        cur_dist = act_dist - dlq_dist
        return act_dist, dlq_dist, cur_dist, def_dist

    def cur_balance_calc(self, model_balance, cur_dist):
        '''
        Расчет модельного баланса в статусе CUR
        :param model_balance: плановый график баланса
        :param cur_dist: доля клиентов в статусе CUR на каждый месяц
        :return:
        principal_balance_cur : np.array(101) : principal balance в статусе CUR
        gross_balance_cur : np.array(101) : gross balance в статусе CUR
        '''
        insurance_cost = model_balance[0] * self.insurance_rate * 0.4 # считаем прибыль со страховки
        principal_balance_cur = (model_balance + insurance_cost)*cur_dist
        gross_balance_cur = (model_balance + insurance_cost) * cur_dist
        return principal_balance_cur, gross_balance_cur

    def dlq_balance_calc(self, model_balance, regular_payment, dlq_dist):
        '''
        Расчет модельного баланса в статусе DLQ
        :param model_balance: плановый график баланса
        :param regular_payment: размер регулярного платежа
        :param dlq_dist: доля клиентов в статусе DLQ на каждый месяц
        :return:
        principal_balance_dlq : np.array(101) : principal balance в статусе DLQ
        gross_balance_dlq : np.array(101) : gross balance в статусе DLQ
        '''

        insurance_cost = model_balance[0] * 0.6 * self.insurance_rate
        principal_balance_dlq = np.append(model_balance[1:], model_balance[-1] + insurance_cost)*dlq_dist
        gross_balance_dlq = (model_balance + regular_payment*self.avg_missed_payments + 2 * insurance_cost)*dlq_dist #дважды добавим insurance: 1
        return principal_balance_dlq, gross_balance_dlq

    def act_balance_calc(self, principal_balance_cur, principal_balance_dlq, gross_balance_cur, gross_balance_dlq):
        '''
        Расчет модельного баланса в статусе ACT
        :param principal_balance_cur: principal balance в статусе CUR
        :param principal_balance_dlq: principal balance в статусе DLQ
        :param gross_balance_cur: principal balance в статусе CUR
        :param gross_balance_dlq: principal balance в статусе DLQ
        :param dlq_dist: доля клиентов в статусе DLQ на каждый месяц
        :return:
        principal_balance_act : np.array(101) : principal balance в статусе ACT
        gross_balance_act : np.array(101) : gross balance в статусе ACT
        '''
        principal_balance_act = principal_balance_cur + principal_balance_dlq
        gross_balance_act = gross_balance_cur + gross_balance_dlq
        return principal_balance_act, gross_balance_act

    def def_balance_calc(self, model_balance, regular_payment, def_dist):
        '''
        Расчет модельного баланса в статусе DEF
        :param model_balance: плановый график баланса
        :param regular_payment: размер регулярного платежа
        :param def_dist: доля клиентов в статусе DEF на каждый месяц
        :return:
        principal_balance_def : np.array(101) : principal balance в статусе DEF
        gross_balance_def : np.array(101) : gross balance в статусе DEF
        '''
        principal_balance_def = np.zeros(101)
        gross_balance_def = np.zeros(101)

        def_dist_change = def_dist[4:] - def_dist[3:-1]
        principal_balance_def[4:] = np.cumsum(model_balance[:-4] * def_dist_change)
        gross_balance_def[4:] = np.cumsum((model_balance[4:] + 4 * regular_payment) * def_dist_change)

        return principal_balance_def, gross_balance_def

    def profit_calc(self, principal_balance_act, principal_balance_def, term, rate, dlq_dist):
        '''

        :param principal_balance_act: principal balance в статусе ACT
        :param principal_balance_def: principal balance в статусе DEF
        :param term: срок кредита
        :param rate: тавка по кредиту
        :param dlq_dist: доля клиентов в статусе DLQ на каждый месяц
        :return:
        profit : np.array(101) : доход на каждый месяц
        '''
        interest_profit = principal_balance_act*rate/12
        penatly_profit = dlq_dist*self.dlq_penalty_amount

        new_def_balance = np.append(principal_balance_def[1:], 0) - principal_balance_def
        recovery_profit = new_def_balance*self.recovery

        profit = interest_profit + recovery_profit + penatly_profit
        profit[term+1:] = 0

        return profit

    def insurance_profit_calc(self, amount, cur_dist):
      return  np.append(0, cur_dist[1:] * self.insurance_rate * amount)

    def insurance_loss_calc(self, cur_dist, dlq_dist, gross_balance_cur, gross_balance_dlq):
      costs = gross_balance_cur[1:] * 0.1/100 * cur_dist[1:] * 0.4 + gross_balance_dlq[1:] * dlq_dist[1:] * 0.1/100 * 0.6 # каждый месяц вероятность 1/1000, что клиент воспользуется страховкой
      return np.append(0, costs)


    def loss_calc(self, gross_balance_act, principal_balance_def, term, act_dist, dlq_dist, def_dist):
        '''

        :param gross_balance_act: gross balance в статусе ACT
        :param principal_balance_def: principal balance в статусе DEF
        :param term: срок кредита
        :param act_dist: доля клиентов в статусе ACT на каждый месяц
        :param dlq_dist: доля клиентов в статусе DLQ на каждый месяц
        :param def_dist: доля клиентов в статусе DEF на каждый месяц
        :return:
        loss : np.array(101) : лосс на каждый месяц
        '''
        oper_loss = np.append(act_dist[1:], 0)*self.oper_costs
        loan_loss =  principal_balance_def - np.append(0, principal_balance_def[:-1])
        collection_loss = (np.append(dlq_dist[1:], 0) + np.append(def_dist[1:], 0))*self.collection_costs

        prev_gross_balance_act = np.append(gross_balance_act[0], gross_balance_act[1:])
        cost_of_funds_loss = prev_gross_balance_act*(1 - self.eq_req)*self.cost_of_funds/12

        loss = loan_loss + cost_of_funds_loss + oper_loss + collection_loss
        loss[term+1:] = 0

        return loss, loan_loss, cost_of_funds_loss, oper_loss, collection_loss

    def assets_liabilities_calc(self, gross_balance_act):
        '''

        :param gross_balance_act: gross balance в статусе ACT
        :return:
        assets : np.array(101) : активы в проекте
        eq_req_curve : np.array(101) : активы, обеспеченные капиталом
        fund_req_curve : np.array(101) : активы, обеспеченные фондами
        '''

        # assets
        assets = gross_balance_act

        #liabilities
        eq_req_curve = assets*self.eq_req
        fund_req_curve = assets*(1 - self.eq_req)

        return assets, eq_req_curve, fund_req_curve

    def niat_calc(self, profit, loss):
        '''
        Расчет NIAT
        :param profit: суммарный доход на каждый месяц
        :param loss: суммарный лосс на каждый месяц
        :return:
        nibt : np.array(101) : прибыль до налогообложения
        niat : np.array(101) : прибыль после налогообложения
        tax : np.array(101) : налог в каждом месяце
        '''

        nibt = profit - loss
        tax = nibt*self.tax_rate
        niat = nibt - tax

        return nibt, niat, tax

    def cashflow_calc(self, principal_balance_act, principal_balance_def, amount, profit,
                      cost_of_funds_loss, oper_costs, collection_costs, tax, niat, eq_req_curve, fund_req_curve):
        '''
        Расчет денежных потоков
        :param principal_balance_act: principal balance в статусе DEF
        :param principal_balance_def: principal balance в статусе DEF
        :param amount: срок кредита
        :param profit: доход на каждый месяц
        :param cost_of_funds_loss: лоссы на фондирование
        :param oper_costs: операционные расходы
        :param collection_costs: расходы на коллекшн
        :param tax: налог в каждом месяце
        :param niat: доход после налогообложения
        :param eq_req_curve: активы, обеспеченные капиталом
        :param fund_req_curve: активы, обеспеченные фондами
        :return:
        cf_to_client : np.array(101) : денежный поток к клиенту
        cf_to_shareholders : np.array(101) : денежный поток к акционерам
        cf_to_debtholders : np.array(101) : денежный поток к фондам
        cf_to_cost_and_tax : np.array(101) : денежный поток на косты и налоги
        '''
        # client
        delta_principal_balance_act = (np.append(0, principal_balance_act[:100]) - principal_balance_act)
        delta_principal_balance_def = (np.append(0, principal_balance_def[:100]) - principal_balance_def)
        repayments = delta_principal_balance_act - delta_principal_balance_def + profit
        cf_to_client = np.append(amount, -repayments[1:])

        # debtoholder
        fund_req_ch = fund_req_curve - np.append(fund_req_curve[0], fund_req_curve[:-1])
        fund_req_change = np.append(fund_req_curve[0], fund_req_ch[1:])
        cf_to_debtholders = cost_of_funds_loss - fund_req_change

        # shareholders
        eq_req_ch = eq_req_curve - np.append(eq_req_curve[0], eq_req_curve[:-1])
        cf_to_shareholders = niat - np.append(eq_req_curve[0], eq_req_ch[1:])

        # cost and tax
        cf_to_cost_and_tax = oper_costs + collection_costs + tax

        return cf_to_client, cf_to_shareholders, cf_to_debtholders, cf_to_cost_and_tax

    def npv_calc(self, pd, pa, amount, rate, term, path_def_rate, path_clo_rate, path_dlq_ratio):
        '''
        Расчет NPV
        :param pd: вероятность дефолта к 12-ому месяцу
        :param pa: вероятность полного досрочного погашения к 6-ому месяцу
        :param amount: сумма кредита
        :param rate: ставка по кредиту
        :param term: срок кредита
        :return: npv: чистая приведенная стоимость кредита
        '''

        # Считаем балансы
        model_balance, interest, regular_payment =  self.model_balance_calculations(amount, rate, term)

        # Считаем распределение
        act_dist, dlq_dist, cur_dist, def_dist = self.distribution_calc(pd, pa, term, path_def_rate, path_clo_rate, path_dlq_ratio)

        # Считаем балансы
        principal_balance_cur, gross_balance_cur = self.cur_balance_calc(model_balance, cur_dist)
        principal_balance_dlq, gross_balance_dlq = self.dlq_balance_calc(model_balance, regular_payment, dlq_dist)
        principal_balance_act, gross_balance_act = self.act_balance_calc(principal_balance_cur, principal_balance_dlq, gross_balance_cur, gross_balance_dlq)
        principal_balance_def, gross_balance_def = self.def_balance_calc(model_balance, regular_payment, def_dist)

        # Считаем профиты
        profit = self.profit_calc(principal_balance_act, principal_balance_def, term, rate, dlq_dist)
        loss, loan_loss, cost_of_funds_loss, oper_loss, collection_loss = self.loss_calc(gross_balance_act, principal_balance_def, term, act_dist, dlq_dist, def_dist)
        # считаем лосс по insurance
        insurance_losses = self.insurance_loss_calc(cur_dist, dlq_dist, gross_balance_cur, gross_balance_dlq)
        #прибавляем к лоссам
        loss += insurance_losses
        # считаем профит по insurance

        insurance_gains = self.insurance_profit_calc(amount, cur_dist)

        #прибавляем к профитам
        profit+=insurance_gains
        nibt, niat, tax = self.niat_calc(profit, loss)

        # Считаем активы и денежные потоки
        assets, eq_req_curve, fund_req_curve = self.assets_liabilities_calc(gross_balance_act)
        cf_to_client, cf_to_shareholders, cf_to_debtholders, cf_to_cost_and_tax =\
            self.cashflow_calc(principal_balance_act, principal_balance_def, amount, profit, cost_of_funds_loss, oper_loss, collection_loss,
                               tax, niat, eq_req_curve, fund_req_curve)

        # Считаем NPV
        disc_curve = np.array([1 / ((1 + self.discounting_rate) ** (i / 12.)) for i in range(101)])
        pv = np.round(np.sum(disc_curve * cf_to_shareholders)).astype(int)
        npv = pv - self.acquisition_cost

        return npv

### Загрузка PA, PD из прошлого датасета


In [4]:
path_def_rate = "/content/def_rate.csv"
path_clo_rate = "/content/clo_rate.csv"
path_dlq_ratio = "/content/dlq_ratio.csv"

In [8]:
model = NPVModel()

model.npv_calc(pd=0.02, pa=0.10, amount=500_000, term=12, rate=0.25, path_def_rate=path_def_rate, path_clo_rate=path_clo_rate, path_dlq_ratio=path_dlq_ratio )

48955